In [1]:
from zipfile import ZipFile

kmz = ZipFile('Functional Pumps.kmz', 'r')
kml = kmz.open('doc.kml', 'r').read()

# Badanie dostępu do wody pitnej

Jednym z najważniejszych obszarów pomocy humaniternej m.in. w Afryce to zapewnianie dostępu do czystej wody pitnej. Niektóre organizacje zajmujące się utrzymywaniem studni słodkowodnych, takie jak [Water For Good](https://waterforgood.org/well-map/) udostępniają mapy zawierające informacje o punktach dostępu do wody, którymi się zajmują. W przypadku braku takich informacji można zastosować algorytmy detekcji obiektów, aby znaleźć studnie na zdjęciach satelitarnych.

In [2]:
from lxml import html

doc = html.fromstring(kml)

## Przetwarzanie danych lokalizacyjnych studni

Plik KMZ zawiera dane w formacie punktów o pewnych współrzędnych geograficznych. W naszym przypadku plik zawiera wyłącznie lokalizacje studni, które zapisujemy do listy.

In [3]:
def parseKMZFile(kmz_file_path: str) -> list:
    """
    Parses Google My Maps KMZ file into a list of pointmarks.
    
    Parameters
    ---
    - `kmz_file_path` - path to KMZ file to be analyzed.

    Returns
    ---
    A list of placemark coordinates in `[latitude, longitude]` format
    """

    unpacked_kmz = ZipFile('Functional Pumps.kmz', 'r')
    map_point_data = unpacked_kmz.open('doc.kml', 'r').read()
    map_point_data_parsed = html.fromstring(map_point_data)

    wells_coordinates = []

    for placemark_coordinate in map_point_data_parsed.cssselect("Document Placemark Point"):
        well_coordinates_raw = placemark_coordinate.text_content()
        well_coordinates_split = well_coordinates_raw.split(',')
        well_coordinates = (
        float(well_coordinates_split[1].strip()),
        float(well_coordinates_split[0].strip())
    )
        wells_coordinates.append(well_coordinates)
    return wells_coordinates

# Znajdowanie dystansu od najbliższej studni

In [8]:
from geopy.distance import distance

def closestWellFromLocation(origin_coordinates: float, wells_coordinates: list) -> tuple:
    """
    Returns the position of the well that is closest to origin point and distance between them.

    All coordinates should be in `[latitude, longitude]` format.
    """
    distances_to_wells = [
        distance(origin_coordinates, well_coordinates).m for well_coordinates in wells_coordinates
    ]
    minimal_distance = min(distances_to_wells)
    index_of_the_closest_well = distances_to_wells.index(minimal_distance)
    closest_well_coordinates = wells_coordinates[index_of_the_closest_well]
    return(
        closest_well_coordinates,
        minimal_distance
    )

In [9]:
random_point_near_wells = [5.74639, 19.12389]

wells_coordinates = parseKMZFile('Functional Pumps.kmz')

closest_well_coordinates, distance_to_well = closestWellFromLocation(random_point_near_wells, wells_coordinates)
closest_well_latitude, closest_well_longitude = closest_well_coordinates
distance_to_well = round(distance_to_well)

print(f"The closest water well is at lat {closest_well_latitude}, lon {closest_well_longitude}; the distance to it is approx. {distance_to_well}m")

The closest water well is at lat 5.734811, lon 19.129088; the distance to it is approx. 1404m
